In [1]:
# for webscraping
from selenium import webdriver 
from selenium.webdriver.common.by import By 
from selenium.webdriver.support.ui import WebDriverWait 
from selenium.webdriver.support import expected_conditions as EC 
from selenium.common.exceptions import TimeoutException
from selenium.webdriver.common.keys import Keys
import time

import numpy as np  # for multi-dimensional containers 
import pandas as pd # for DataFrames
import re # regex

In [2]:
def xpath_wait(x_path,time):
    '''
    handles wait for loading elements found by xpath
    '''
    wait = WebDriverWait(driver, time)
    return wait.until(EC.element_to_be_clickable((By.XPATH,
            x_path)))

def class_wait(class_name,time):
    '''
    handles wait for loading elements found by class name
    '''
    wait = WebDriverWait(driver, time)
    return wait.until(EC.element_to_be_clickable((By.CLASS_NAME,
            class_name)))

In [3]:
# set up driver options
option = webdriver.ChromeOptions()
option.add_argument('--no-sandbox')
option.add_argument("--disable-setuid-sandbox")
option.add_argument("--start-maximized")

In [4]:
base = 'https://www.trackalytics.com/instagram/profile/'
profiles = [base + 'barackobama/', base + 'berniesanders/', 
            base + 'joebiden/', base + 'elizabethwarren/',
            base + 'realdonaldtrump/', base + 'sentedcruz/']

In [60]:
columns = ['name','date','followers','followers_change']
df1 = pd.DataFrame(columns=columns)
df1

,name,date,followers,followers_change


In [9]:
driver = webdriver.Chrome('C:/Users/tyler/chromedriver_win32/chromedriver.exe', options = option)

In [76]:
for p in profiles:
    driver.get(p)
    xpath_wait('//*[@id="summary_length"]/label/select/option[5]',3).click() # open last 500 days of followers

    # get name
    name = p.split('profile/')[1].replace('/','')
    
    # get all text from table
    posts = driver.find_elements_by_tag_name('td')
    
    # lists to hold data
    names = [name] * (len(posts)//6)
    dates = []
    followers = []
    followers_change = []

    # obtain useful data
    for i in range(len(posts)//6):
        # indices of text in table
        date_idx = 6 * i + 1
        followers_idx = 6 * i + 2

        # format and append data
        dates.append(posts[date_idx].text)
        follower_count,follower_change = posts[followers_idx].text.replace(',','').split(' - ')
        follower_change = follower_change.replace('(','').replace(')','').replace('+','')
        followers.append(follower_count)
        followers_change.append(follower_change)

    # turn into dataframe and append
    data = {'name':names, 'date':dates,'followers':followers,'followers_change':followers_change}
#     print(name + str(len(posts)) + ' ' + str(len(names)) + ' ' + str(len(dates)) + ' ' + str(len(followers)) + ' ' + str(len(followers_change)))
    df2 = pd.DataFrame(data)
    df1 = df1.append(df2)

In [78]:
df1.to_csv('poli_ig_followers.csv',index=False)